# Air Quality and COVID-19 Infection Rates
For this project, our goal is to investigate a possible relationship between Air Quality and COVID-19 Infection Rates. To that end, we will merge an Air Quality dataset with a COVID-19 dataset, train a linear regression model on the merged dataset, and perform predictions to test the accuracy of our model.

Some thoughts about the web app:
- Perhaps an app where Air Quality features and COVID-19 cases are the independent variables while COVID-19 infection rate (%) is the dependent variable.
- Sliders and input fields would be used for the independent variables.

**Important:** Because this is an exploratory project, we are not absolutely certain that there exists a significant relationship between Air Quality and COVID-19 datasets. Furthermore, our results, significant or not, can only lend support to hypotheses surrounding Air Quality and COVID-19.

For full project motivations, please see `CS180_G16_ProjectProposal.pdf`.